In [1]:
import logging
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import sklearn

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Air-BnB-2-BW/data-science/master/airbnb_BW2.csv')

In [3]:
dataset = df.values

In [4]:
X = dataset[:,0:12]
y = dataset[:,12]
y = np.reshape(y, (-1,1))
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

In [5]:
scaler_x.fit(X)
xscale=scaler_x.transform(X)
scaler_y.fit(y)
yscale=scaler_y.transform(y)

In [6]:
X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y.reshape(len(y),1))[:,0]
X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

In [7]:
model = Sequential()
model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=50, batch_size=50,  verbose=1, validation_split=0.2)

Epoch 1/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2283 - mse: 0.2283 - mae: 0.4471 - val_loss: 0.2128 - val_mse: 0.2128 - val_mae: 0.4301
Epoch 2/50
1014/1014 [==============================] - 1s 1ms/step - loss: 0.2125 - mse: 0.2125 - mae: 0.4259 - val_loss: 0.2114 - val_mse: 0.2114 - val_mae: 0.4244
Epoch 3/50
1014/1014 [==============================] - 1s 989us/step - loss: 0.2115 - mse: 0.2115 - mae: 0.4226 - val_loss: 0.2104 - val_mse: 0.2104 - val_mae: 0.4233
Epoch 4/50
1014/1014 [==============================] - 1s 874us/step - loss: 0.2111 - mse: 0.2111 - mae: 0.4219 - val_loss: 0.2119 - val_mse: 0.2119 - val_mae: 0.4277
Epoch 5/50
1014/1014 [==============================] - 1s 946us/step - loss: 0.2107 - mse: 0.2107 - mae: 0.4209 - val_loss: 0.2106 - val_mse: 0.2106 - val_mae: 0.4207
Epoch 6/50
1014/1014 [==============================] - 1s 866us/step - loss: 0.2107 - mse: 0.2107 - mae: 0.4208 - val_loss: 0.2109 - val_mse: 0.2109 - val_mae: 0.4

In [8]:
model.save('keras_nn_model.h5')